<a href="https://colab.research.google.com/github/npr99/PlanningMethods/blob/master/Location_Quotient_Using_BLS_QCEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Location Quotient Example Using BLS QCEW

The Bureau of Labor Statistics provides employment and wage data by year, county, state, and nation by industry.

An overview of this data is available on the BLS website for the Quarterly Census of Employment and Wages (QCEW) data

https://www.bls.gov/cew/

The BLS provides data in "data slices" - for individual states and counties.

https://data.bls.gov/cew/doc/access/csv_data_slices.htm

The BLS provides a complete layout of the datasets - specifically for Location Quotient the Annual Average data is a good choice:

https://data.bls.gov/cew/doc/access/csv_data_slices.htm#ANNUAL_LAYOUT

The BLS provides a complete list of Areas (states and counties) - the "area codes" are based on the state and county FIPS codes:

https://data.bls.gov/cew/doc/titles/area/area_titles.htm

### Example Area Slice Files

2016 QCEW Annual averages for Georgia
http://data.bls.gov/cew/data/api/2016/a/area/13000.csv

2016 QCEW Annual averages for Chatham County, GA
http://data.bls.gov/cew/data/api/2016/a/area/13051.csv

### File Layout
https://www.bls.gov/cew/about-data/downloadable-file-layouts/annual/naics-based-annual-layout.htm

#### Details on codes for different ownership types:
https://www.bls.gov/cew/classifications/ownerships/ownership-titles.htm


Note totals for employment are the sums of all Annual average of monthly employment levels for a given year (annual_avg_emplvl) for all ownership codes.

### BLS data as time series
Chatham County QCEW Time Series Data
https://data.bls.gov/timeseries/ENU1305110010


## Step 1: Obtain BLS QCEW Data File
The Pandas read csv command is a fast way to download .csv datafiles directly to the notebook session.

In [19]:
import pandas as pd # For reading, writing and wrangling data

blsqcew = pd.read_csv('http://data.bls.gov/cew/data/api/2016/a/area/13051.csv')
blsqcew.head()

,area_fips,own_code,industry_code,agglvl_code,size_code,year,qtr,disclosure_code,annual_avg_estabs,annual_avg_emplvl,total_annual_wages,taxable_annual_wages,annual_contributions,annual_avg_wkly_wage,avg_annual_pay,lq_disclosure_code,lq_annual_avg_estabs,lq_annual_avg_emplvl,lq_total_annual_wages,lq_taxable_annual_wages,lq_annual_contributions,lq_annual_avg_wkly_wage,lq_avg_annual_pay,oty_disclosure_code,oty_annual_avg_estabs_chg,oty_annual_avg_estabs_pct_chg,oty_annual_avg_emplvl_chg,oty_annual_avg_emplvl_pct_chg,oty_total_annual_wages_chg,oty_total_annual_wages_pct_chg,oty_taxable_annual_wages_chg,oty_taxable_annual_wages_pct_chg,oty_annual_contributions_chg,oty_annual_contributions_pct_chg,oty_annual_avg_wkly_wage_chg,oty_annual_avg_wkly_wage_pct_chg,oty_avg_annual_pay_chg,oty_avg_annual_pay_pct_chg
0,13051,0,10,70,0,2016,A,NaN,8654,149090,6613717155,1354478311,22797107,853,44361,NaN,1.00,1.00,1.00,1.0,1.0,1.00,1.00,NaN,277,3.3,3627,2.5,155753462,2.4,40394517,3.1,-1835966,-7.5,-1,-0.1,-35,-0.1
1,13051,1,10,71,0,2016,A,NaN,64,2603,184370024,0,0,1362,70837,NaN,1.19,0.89,0.97,0.0,0.0,1.09,1.09,NaN,0,0.0,32,1.2,2698690,1.5,0,0.0,0,0.0,3,0.2,180,0.3
2,13051,1,102,72,0,2016,A,NaN,64,2603,184370024,0,0,1362,70837,NaN,1.19,0.90,0.99,0.0,0.0,1.09,1.09,NaN,0,0.0,32,1.2,2698690,1.5,0,0.0,0,0.0,3,0.2,180,0.3
3,13051,1,1021,73,0,2016,A,NaN,15,498,30001695,0,0,1159,60275,NaN,0.56,0.70,0.85,0.0,0.0,1.22,1.22,NaN,0,0.0,8,1.6,-509113,-1.7,0,0.0,0,0.0,-38,-3.2,-1971,-3.2
4,13051,1,1023,73,0,2016,A,NaN,2,17,1615844,0,0,1828,95050,NaN,2.81,1.23,1.26,0.0,0.0,1.03,1.03,NaN,0,0.0,-1,-5.6,-58322,-3.5,0,0.0,0,0.0,48,2.7,2469,2.7


## Step 3: Explore Data
Look at descripitive statistics for key variabiables.

In [7]:
# Explore the Location quotient of annual average employment relative to the U.S. (Rounded to the hundredths place)
blsqcew[['annual_avg_estabs','annual_avg_emplvl','lq_annual_avg_emplvl','lq_total_annual_wages',]].describe()

,annual_avg_estabs,annual_avg_emplvl,lq_annual_avg_emplvl,lq_total_annual_wages
count,1703.000000,1703.000000,1703.000000,1703.000000
mean,45.806812,702.179683,0.766753,0.771321
std,365.106190,5845.989363,2.602384,2.100285
min,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.000000
50%,5.000000,13.000000,0.160000,0.140000
75%,15.000000,197.000000,0.940000,0.950000
max,8654.000000,149090.000000,56.370000,35.960000


Look at top Location Quotients by industry

In [8]:
blsqcew[['industry_code','annual_avg_estabs','annual_avg_emplvl','lq_annual_avg_emplvl','lq_total_annual_wages']].sort_values(by='lq_annual_avg_emplvl', ascending=False).head(10)

,industry_code,annual_avg_estabs,annual_avg_emplvl,lq_annual_avg_emplvl,lq_total_annual_wages
1028,488320,13,3581,56.37,35.96
1027,48832,13,3581,56.37,35.96
1024,4883,28,3823,39.80,26.33
47,621493,1,124,23.32,26.21
1006,4871,10,243,16.52,14.20
1007,48711,10,243,16.52,14.20
1008,487110,10,243,16.52,14.20
120,62142,13,196,13.71,14.10
121,621420,13,196,13.71,14.10
1029,48833,7,181,10.13,10.45


## Full list of industry codes
For a full list of industry codes use the link below:

https://www.bls.gov/cew/classifications/industry/industry-titles.htm


## More details on NAICS
North American Industry Classification System (NAICS) 

https://www.naics.com/search/

## NAICS FAQ
https://www.naics.com/frequently-asked-questions/#NAICSfaq

Look at top industries by employement.

In [9]:
blsqcew[['industry_code','annual_avg_estabs','annual_avg_emplvl','lq_annual_avg_emplvl','lq_total_annual_wages']].sort_values(by='annual_avg_emplvl', ascending=False).head(10)

,industry_code,annual_avg_estabs,annual_avg_emplvl,lq_annual_avg_emplvl,lq_total_annual_wages
0,10,8654,149090,1.00,1.00
223,10,8527,130796,1.03,1.01
228,102,7677,110314,1.05,0.92
229,1021,2126,35113,1.24,1.26
234,1026,1103,23591,1.44,1.48
233,1025,901,23336,1.03,1.29
1572,72,975,21645,1.55,1.72
224,101,850,20483,0.93,1.38
1431,62,822,19895,1.00,1.26
791,44-45,1295,18771,1.13,1.24


Look at summary data by 2-digit NAICS code and Ownership Code.

In [30]:
blsqcew_2digit = blsqcew[['industry_code','own_code','annual_avg_estabs','annual_avg_emplvl']].\
loc[(blsqcew['industry_code'].str.len()==2) |
    (blsqcew['industry_code'].str.contains("-"))].sort_values(by=['industry_code','own_code'])
blsqcew_2digit.head()

,industry_code,own_code,annual_avg_estabs,annual_avg_emplvl
0,10,0,8654,149090
1,10,1,64,2603
98,10,2,49,4774
182,10,3,16,10917
223,10,5,8527,130796


In [31]:
#  Need to drop total covered - own_code == 0 - leads to double counting
blsqcew_2digit = blsqcew_2digit.loc[(blsqcew['own_code'] !=0)]
blsqcew_2digit.head()

,industry_code,own_code,annual_avg_estabs,annual_avg_emplvl
1,10,1,64,2603
98,10,2,49,4774
182,10,3,16,10917
223,10,5,8527,130796
237,11,5,15,0


In [32]:
# Replace Industry code for NAICS 92 to get levels of government
blsqcew_2digit.loc[(blsqcew['industry_code'] =='92')]

,industry_code,own_code,annual_avg_estabs,annual_avg_emplvl
57,92,1,39,1403
135,92,2,27,1912
202,92,3,13,5346


In [33]:
blsqcew_2digit.loc[(blsqcew['industry_code'] =='92') &
                   (blsqcew['own_code'] ==1),'industry_code'] = '92 1 Federal Government'
blsqcew_2digit.loc[(blsqcew['industry_code'] =='92') &
                   (blsqcew['own_code'] ==2),'industry_code'] = '92 2 State Government'
blsqcew_2digit.loc[(blsqcew['industry_code'] =='92') &
                   (blsqcew['own_code'] ==3),'industry_code'] = '92 3 Local Government'

In [34]:
blsqcew_2digit.groupby(by="industry_code").sum()

,own_code,annual_avg_estabs,annual_avg_emplvl
industry_code,,,
10,11,8656,149090
11,5,15,0
21,5,2,0
22,5,14,336
23,5,603,5466
31-33,5,231,14913
42,5,391,5442
44-45,6,1296,18816
48-49,8,442,11019
